In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import gzip
import os
import json
import gzip

C:\Users\chris\anaconda4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\chris\anaconda4\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\chris\anaconda4\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
restaurants = pd.read_csv('restaurants.csv')
ratings = pd.read_csv('ratings.csv')
df = restaurants.merge(ratings, how="left", on="RestaurantId")
df.head()

,RestaurantId,Name,UserId,Review,Rating
0,1,aleria,1,A full 5 stars! Phenomenal Michelin ranked res...,5
1,1,aleria,2,We spent great evening here. Cozy restaurant w...,5
2,1,aleria,3,10/10. Aleria was our highlight during our tri...,5
3,1,aleria,4,Great service and fantastic food. We enjoyed e...,5
4,1,aleria,5,Let me put it his way: simply perfect. We hate...,5


In [ ]:
del df['Rating']
del df['RestaurantId']
del df['UserId']
df

,Name,Review
0,aleria,A full 5 stars! Phenomenal Michelin ranked res...
1,aleria,We spent great evening here. Cozy restaurant w...
2,aleria,10/10. Aleria was our highlight during our tri...
3,aleria,Great service and fantastic food. We enjoyed e...
4,aleria,Let me put it his way: simply perfect. We hate...
...,...,...
26203,ydria,"Very good grilled octopus, spinach pie and Sau..."
26204,ydria,"The food is sensarional, the service is great...."
26205,ydria,Risotto was excellent. Wine was an extremely s...
26206,ydria,We decided to eat here for an afternoon snack ...


In [ ]:
df = df.groupby(['Name'])['Review'].apply(','.join).reset_index()
df

,Name,Review
0,aleria,A full 5 stars! Phenomenal Michelin ranked res...
1,alexander_the_great,Excellent food and service. \n\nI have been co...
2,arcadia,Nice restaurant with indoor and outdoor seatin...
3,athenascook,"Πολύ νόστιμο και εξαιρετικής ποιότητας φαγητό,..."
4,atitamos,"Είναι τρομερό, το βρήκα μια μέρα τυχαία που το..."
5,balcony,We were looking for somewhere to eat outside a...
6,dio_dekares,Εξαιρετικό εστιατόριο καθ όλα. Φαγητό με γεύσε...
7,efcharis,"Excellent place to eat. The prices are good, f..."
8,elaea,Service was impeccable by the staff of Elaea M...
9,gods_restaurant,They have great food !! Excelent service!! A ...


In [ ]:
# Κατασκευή ενός pipeline για την εξαγωγή βαθμολογιών λέξεων TF-IDF από το δεδομένο κείμενο χωρίς λέξεις τερματισμού (stopwords)
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,max_features=1000, stop_words= ENGLISH_STOP_WORDS)),('model',KMeans(n_clusters=20))])

In [ ]:
# Κάνουμε fit το μοντέλο και το εφαρμόζουμε στα reviews, με αποτέλεσμα να τα κατατάσει σε clusters.
model = pipeline.fit(df['Review'])
a = model.predict(df['Review'])
df['Prediction'] = a 

In [ ]:
df.iloc[:,[0,2]]

,Name,Prediction
0,aleria,3
1,alexander_the_great,7
2,arcadia,19
3,athenascook,0
4,atitamos,0
5,balcony,10
6,dio_dekares,0
7,efcharis,12
8,elaea,6
9,gods_restaurant,7


In [ ]:
# φτιάχνουμε ένα lookup table όπου αποθηκεύουμε κάθε εστιατόριο με το Id του, το ονομάοζουμε lkup_tbl
lkup_tbl = df.iloc[:,[0,2]]
lkup_dict = dict(zip(lkup_tbl.Name, lkup_tbl.Prediction))
lkup_dict

{'aleria': 3,
 'alexander_the_great': 7,
 'arcadia': 19,
 'athenascook': 0,
 'atitamos': 0,
 'balcony': 10,
 'dio_dekares': 0,
 'efcharis': 12,
 'elaea': 6,
 'gods_restaurant': 7,
 'karamanlidika': 1,
 'klimataria': 18,
 'liondi': 7,
 'lithos': 5,
 'maiandros': 12,
 'mani_mani': 0,
 'mironi': 15,
 'nolan': 8,
 'o_thanasis': 4,
 'oroscopo': 13,
 'rozalia': 1,
 'scholarchio': 17,
 'sin_athina': 14,
 'smile': 7,
 'strofi': 2,
 'the_greco': 0,
 'to_kati_allo': 16,
 'to_paradosiako': 9,
 'tzitzikas_kai_mermigas': 1,
 'ydria': 11}

In [ ]:
# θέτουμε ένα text, το οποίο θεωρείται ως input
text = ["warm rooms","polite service, perfectly cooked food"]

# προβλέπουμε σε πιο cluster ανοίκει το input text με την χρήση του μοντέλου
model = pipeline.predict(text)
model

array([7, 7])

In [ ]:
# απαραίτητες βιβλιοθήκες για να αποθηκεύσουμε το look up table και το μοντέλο
from joblib import load
from joblib import dump

In [ ]:
# αποθήκευση του pipeline
dump(pipeline, filename="text_classification1.joblib")

['text_classification1.joblib']

In [ ]:
# κατάλληλες βιβλιοθήκες για την φόρτωση του μοντέλου
import pickle
from joblib import load

In [ ]:
example_dict = lkup_dict

pickle_out = open("dict.pickle1","wb")
pickle.dump(example_dict, pickle_out)
pickle_out.close()

In [ ]:
# συνάρτηση που κάνει recommend
def recommend(text):
  pipeline = load("text_classification1.joblib")
  pickle_in = open("dict.pickle1","rb")
  example_dict = pickle.load(pickle_in)
  out_pred = pipeline.predict(text)
  list_recomm = [id for id, pred in example_dict.items() if pred == out_pred][:20]
  return list_recomm

In [ ]:
recommend(["Eggs were amazing!"])

['efcharis', 'maiandros']